In [8]:
# useful additional packages 
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
#provider = IBMQ.load_account()
# DOcplex
from docplex.mp.model import Model
# Aqua
from qiskit import BasicAer
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.components.optimizers import SPSA
from qiskit.aqua.components.variational_forms import RY
from qiskit.aqua import QuantumInstance
#from qiskit.optimization.ising import docplex, max_cut, tsp
#from qiskit.optimization.ising.common import sample_most_likely
# setup aqua logging
import logging
from qiskit.aqua import set_qiskit_aqua_logging
set_qiskit_aqua_logging(logging.DEBUG)  # choose INFO, DEBUG to see the log
#provider = IBMQ.get_provider(hub='ibm-q')

In [ ]:
# Create an instance of a model and variables
mdl = Model(name='bin_pack')
x = {i: mdl.binary_var(name='x_{0}'.format(i)) for i in range(1,4)}

# Objective function
max_vars_func = mdl.sum(x[i] for i in range(1,4))
mdl.maximize(max_vars_func)

# Constraints
weight = [2, 4, 6]
mdl.add_constraint(mdl.sum(weight[i-1]*x[i] for i in range(1,4)) == 10)

print(mdl.export_to_string())

In [3]:
qubitOp, offset = docplex.get_operator(mdl)
print (qubitOp)
print (offset)

Representation: paulis, qubits: 3, size: 6
118.5


In [4]:
# checking the full Hamiltonian
ee = ExactEigensolver(qubitOp, k=1)
result = ee.run()

print('energy:', result['energy'])
print('objective:', result['energy'] + offset)

x = sample_most_likely(result['eigvecs'][0])
print('solution:', x)

energy: -120.5
objective: -2.0
solution: [0. 1. 1.]


In [5]:
# Full Statevector simulation (MIPexactMAP)
#seed = 10598
seed = 1

spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa)

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('solution objective:', result['energy'] + offset)
print('solution:', x)
print(result)

energy: -120.48148078754409
time: 4.334877252578735
solution objective: -1.9814807875440863
solution: [0. 1. 1.]
{'num_optimizer_evals': None, 'min_val': -120.48148078754409, 'opt_params': array([ 1.72390185,  1.64403167,  0.71614046,  1.41973847, -4.01721689,
        0.36045418, -3.94155515,  0.72714855, -0.00636336,  0.83398437,
       -3.44685059, -1.9060027 ,  3.29793177, -0.2106894 ,  2.40307241,
        1.34569963, -5.08011002,  1.44415837]), 'eval_time': 4.334877252578735, 'eval_count': 651, 'energy': -120.48148078754409, 'eigvals': array([-120.48148079]), 'min_vector': array([ 6.29602228e-04+1.29466385e-16j, -1.19045154e-03+3.97117818e-17j,
       -4.92670378e-04-8.87605551e-17j, -2.47561768e-05+2.63741313e-17j,
       -9.98370098e-04+6.87814360e-17j, -1.22638933e-02-3.26060207e-17j,
        9.99406790e-01-1.15429117e-15j,  3.21342749e-02-1.29868182e-16j]), 'eigvecs': array([[ 6.29602228e-04+1.29466385e-16j, -1.19045154e-03+3.97117818e-17j,
        -4.92670378e-04-8.87605551e-1

In [12]:
# Statevector simulation (MIP SolverSpaceSampling)
#seed = 10598
seed = 1

spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa)

backend = BasicAer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

result = vqe.run(quantum_instance)

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('solution objective:', result['energy'] + offset)
print('solution:', x)
print(result)

energy: -113.92654666022976
time: 6.5045812129974365
solution objective: 4.573453339770239
solution: [0. 1. 1.]
{'num_optimizer_evals': None, 'min_val': -113.92654666022976, 'opt_params': array([-1.22778705,  1.94079957, -1.19201377,  0.84128221, -3.8280963 ,
        0.21097039, -6.73422977, -4.79940094, -0.66815257,  0.35105649,
       -1.17751074, -2.1377842 ,  5.66695976,  0.44269958,  0.03681653,
        2.52529099,  0.49325155, -1.70143691]), 'eval_time': 6.5045812129974365, 'eval_count': 651, 'energy': -113.92654666022976, 'eigvals': array([-113.92654666]), 'min_vector': array([-3.08395802e-04-3.33264438e-17j, -1.79133470e-04-8.35247054e-18j,
       -1.59078700e-02-1.04363182e-16j,  6.75205282e-03+5.16163964e-18j,
        2.46465827e-03+2.43949632e-17j,  5.50511668e-01-5.50789203e-16j,
        7.64420906e-01-8.52477935e-16j, -3.35101075e-01+4.60467304e-16j]), 'eigvecs': array([[-3.08395802e-04-3.33264438e-17j, -1.79133470e-04-8.35247054e-18j,
        -1.59078700e-02-1.04363182e-1

In [10]:
IBMQ.providers()    # List all available providers
IBMQ.get_provider(hub='ibm-q')
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_vigo') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_ourense') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_london') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_burlington') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_essex') from IBMQ(hub='ibm-q', group='open', project='main')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q', group='open', project='main')>]

In [11]:
# run quantum algorithm with shots
#seed = 10598
seed = 1

spsa = SPSA(max_trials=300)
ry = RY(qubitOp.num_qubits, depth=5, entanglement='linear')
vqe = VQE(qubitOp, ry, spsa)

#backend = BasicAer.get_backend('qasm_simulator')
backend = provider.get_backend('ibmq_ourense')
quantum_instance = QuantumInstance(backend, shots=1024, seed_simulator=seed, seed_transpiler=seed, skip_qobj_validation=False)

result = vqe.run(quantum_instance)

x = sample_most_likely(result['eigvecs'][0])
print('energy:', result['energy'])
print('time:', result['eval_time'])
print('solution objective:', result['energy'] + offset)
print('solution:', x)

KeyboardInterrupt: 

In [ ]:
print("result:", result)
print("eigvec:", result['eigvecs'])
plot_histogram(result['eigvecs'])
#print("circuit_width", result['circuit_info']['width'])
#print("circuit_depth", result['circuit_info']['depth'])

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright